In [2]:
import QuantLib as ql

In [14]:
# we define a CommonVars class for creating all variables or arguments that we need in below case.
class swCommonVars:

    def __init__(self):
        self.settlementDays = 2
        self.nominal = 1000000.0
        self.fixedConvention = ql.Unadjusted
        self.fixedFrequency = ql.Annual
        self.fixedDayCount = ql.Thirty360(ql.ModifiedFollowing)

        self.termStructure = ql.RelinkableYieldTermStructureHandle()
        self.index = ql.Euribor6M(self.termStructure)
        self.floatingConvention = self.index.businessDayConvention()
        self.floatingTenor = self.index.tenor()
        self.calendar = self.index.fixingCalendar()
        today = ql.Date(13,3,2002)
        ql.Settings.instance().evaluationDate = today
        self.settlement = self.calendar.advance(today, self.settlementDays, ql.Days)
        self.termStructure.linkTo(ql.FlatForward(self.settlement, 0.05, ql.Actual365Fixed()))

    def makeSwaption(self, swap, exercise, volatility, settlementType):
        vol = ql.QuoteHandle(ql.SimpleQuote(volatility))
        engine = ql.BlackSwaptionEngine(self.termStructure, vol)
        swopt = ql.Swaption(swap, ql.EuropeanExercise(exercise), settlementType)
        swopt.setPricingEngine(engine)
        return swopt
    

In [15]:
# Swaption test case
swVars = swCommonVars()

swVars.termStructure.linkTo(ql.FlatForward(swVars.settlement, 0.05, ql.Actual365Fixed()))
exerciseDate = swVars.calendar.advance(swVars.settlement, ql.Period(5, ql.Years))
startDate = swVars.calendar.advance(exerciseDate, swVars.settlementDays, ql.Days, ql.Following)
forwardStart = ql.Period('0D')
swap = ql.MakeVanillaSwap(ql.Period(10, ql.Years), swVars.index, 0.06, forwardStart, effectiveDate=startDate)
swaption = swVars.makeSwaption(swap, exerciseDate, 0.2, ql.Settlement.Physical)

print(f"test swaption NPV : {swaption.NPV()}")

test swaption NPV : 0.03641815857863785
